In [1]:
import warnings
warnings.filterwarnings("ignore")
import json
import codecs
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import editdistance
import tensorflow as tf
import keras
from gensim.models import Word2Vec
from string import ascii_letters
from string import printable

Using TensorFlow backend.


In [126]:
def preprocessing(js):
    temp = []
    final = []
    for i in range(len(js[list(js.keys())[0]])):
        for j in range(len(js[list(js.keys())[0]][i])):
            if js[list(js.keys())[0]][i][j]['type'] == 'Text':
                temp = temp + [(k,0) for k in js[list(js.keys())[0]][i][j]['value'].split(" ") if k != '']
            else:
                temp = temp + [(k,js[list(js.keys())[0]][i][j]['slot']) for k in js[list(js.keys())[0]][i][j]['value'].split(" ") if k != '']

        final.append(temp)
        temp = []
        
    return final

In [127]:
js=json.load(codecs.open('all_classes.json', 'r', 'utf-8-sig'))
check_all_classes = preprocessing(js)

In [128]:
js=json.load(codecs.open('cehck_for_reminders.json', 'r', 'utf-8-sig'))
check_for_reminders = preprocessing(js)

In [129]:
js=json.load(codecs.open('check_assignment_due.json', 'r', 'utf-8-sig'))
check_assignment_due = preprocessing(js)

In [130]:
js=json.load(codecs.open('find_whether_a_class.json', 'r', 'utf-8-sig'))
check_when_class_is = preprocessing(js)

In [131]:
js=json.load(codecs.open('find_what_is_particular_class.json', 'r', 'utf-8-sig'))
check_what_lecture = preprocessing(js)

In [132]:
js=json.load(codecs.open('reminder_add.json', 'r', 'utf-8-sig'))
add_reminder = preprocessing(js)

In [133]:
js=json.load(codecs.open('where_class.json', 'r', 'utf-8-sig'))
where_class = preprocessing(js)

In [134]:
final_data = check_all_classes + check_for_reminders + check_assignment_due + check_when_class_is + check_what_lecture + add_reminder + where_class

In [135]:
max_len = len(max(final_data,key = len))

In [136]:
'''for i in range(len(final_data)):
    for j in range(max_len-len(final_data[i])):
        final_data[i] = final_data[i] + [("pad_word",0)]'''

'for i in range(len(final_data)):\n    for j in range(max_len-len(final_data[i])):\n        final_data[i] = final_data[i] + [("pad_word",0)]'

In [137]:
for i in range(len(final_data)):
    for j in range(len(final_data[i])):
        if final_data[i][j][1] == 'count':
            final_data[i][j] = (final_data[i][j][0],1)
        elif final_data[i][j][1] == 'day':
            final_data[i][j] = (final_data[i][j][0],2)
        elif final_data[i][j][1] == 'subject':
            final_data[i][j] = (final_data[i][j][0],3)

In [138]:
train = []
target = []

temp0 = []
temp1 = []

for i in range(len(final_data)):
    for j in range(len(final_data[i])):
        temp0 = temp0 + [final_data[i][j][0]]
        temp1 = temp1 + [final_data[i][j][1]]
    train.append(temp0)
    target.append(temp1)
    temp0 = []
    temp1 = []

In [139]:
for i in range(len(train)):
    for j in range(10-len(train[i])):
        train[i] = train[i] + ["pad"]
        target[i] = target[i] + [0]

In [140]:
len(list(printable))

100

In [141]:
count = range(0,len(list(printable)))
vocab_dict = dict(zip(list(printable),count))

In [142]:
vocab_dict['char_pad'] = 100

In [143]:
for i in range(len(train)):
    for j in range(len(train[i])):
        embd = [vocab_dict[i] for i in list(train[i][j])]
        temp = np.zeros([len(embd),len(vocab_dict)],dtype=np.int32)
        for k in range(len(embd)):
            temp[k][embd[k]] =1
        train[i][j] = temp

In [243]:
for i in range(len(target)):
    for j in range(len(target[i])):
        if target[i][j] == 0:
            target[i][j] = [0,0,0,0]
        elif target[i][j] == 1:
            target[i][j] = [0,1,0,0]
        elif target[i][j] == 2:
            target[i][j] = [0,0,1,0]
        elif target[i][j] == 3:
            target[i][j] = [0,0,0,1]

In [244]:
x_train,x_test,y_train,y_test = train_test_split(train,target,test_size = 0.2,random_state = 1234)

In [227]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D

In [229]:
inp = keras.layers.Input(shape=[None,10,101])

char_enc = TimeDistributed(LSTM(units=32, return_sequences=False,
                                recurrent_dropout=0.5))(inp)

lstm = keras.layers.LSTM(32,return_sequences=True)(char_enc)
out = keras.layers.TimeDistributed(keras.layers.Dense(4,activation='softmax'))(lstm)
model = keras.Model(inputs = inp,outputs=out)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, None, 10, 101)     0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 32)          17152     
_________________________________________________________________
lstm_5 (LSTM)                (None, None, 32)          8320      
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 4)           132       
Total params: 25,604
Trainable params: 25,604
Non-trainable params: 0
_________________________________________________________________


In [251]:
item = 0
while item < len(x_train):
    temp = x_train[item]
    feed_target = y_train[item]
    
    feed_target = np.array(feed_target).reshape([1,10,4])
    
    for k in range(len(temp)):
        for j in range(10-len(temp[k])):
            temp[k] = np.vstack((temp[k],char_pad))
    
    temp1 = np.zeros([1,10,10,101])
    
    for i in range(10):
        temp1[0][i] = temp[i]
    
    print(model.train_on_batch(temp1,feed_target))
    
    item += 1
    

[0.0007363132, 0.3]
[0.00075378746, 0.2]
[3.4397795e-05, 0.1]
[7.79254e-05, 0.2]
[6.7477886e-05, 0.2]
[0.0038311332, 0.2]
[0.003066896, 0.2]
[0.0008694728, 0.1]
[4.471944e-05, 0.1]
[0.0007558734, 0.2]
[0.00072739966, 0.3]
[0.0007422495, 0.2]
[0.011816174, 0.2]
[0.00066931365, 0.2]
[0.0031434796, 0.2]
[3.499405e-05, 0.1]
[0.0013744807, 0.3]
[0.008798795, 0.2]
[6.68997e-05, 0.2]
[0.023257896, 0.3]
[0.0011109214, 0.3]
[0.00076219055, 0.2]
[0.04209716, 0.3]
[0.00064401055, 0.2]
[0.000687469, 0.2]
[0.002039664, 0.2]
[0.0011566242, 0.2]
[0.001394741, 0.2]
[0.000683388, 0.2]
[9.907623e-05, 0.1]
[0.011503984, 0.3]
[0.0011305892, 0.3]
[2.5913498e-05, 0.1]
[0.0010758212, 0.2]
[0.0006969583, 0.2]
[9.945277e-05, 0.2]
[0.000987306, 0.3]
[0.0007561719, 0.3]
[0.0008815896, 0.2]
[0.0009010992, 0.2]
[0.0015352515, 0.2]
[0.0006668364, 0.2]
[0.0006187983, 0.2]
[0.0016815823, 0.2]
[0.015697803, 0.2]
[0.00086289534, 0.3]
[0.0010122126, 0.2]
[0.00088720565, 0.3]
[0.0009252823, 0.3]
[0.00080428657, 0.3]
[0.0

KeyboardInterrupt: 